In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib as plb
import seaborn as sns
from scipy import stats
import statistics
import dabest as db

#### 1. Unblinding the metadata
<p>We 

In [2]:
p1 = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1fuPlate1.csv')
p1s = p1.iloc[range(6)]
p1s = p1s.melt(id_vars=['Let'], var_name= 'Col', value_name= 'Compound')
p1s['CPID'] = 'P1'

In [3]:
p2 = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1fuPlate2.csv')
p2s = p2.iloc[range(6)]
p2s = p2s.melt(id_vars=['Let'], var_name= 'Col', value_name= 'Compound')
p2s['CPID'] = 'P2'

In [4]:
sub = ['Compound', 'Location']
ckey = pd.read_csv('C:/Users/Emily/Documents/S1F1/ckey.csv', index_col=0).drop(columns=['Let','Col'])
ckey_x = pd.read_csv('C:/Users/Emily/Documents/S1F1/D5P1.csv', usecols=sub)
ckey_x.rename(columns={'Location': 'Compound Well'}, inplace=True)
ckey_x['CPID'] = 'D5P1'
ckeys = ckey.append(ckey_x)

In [5]:
s1f1_md = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1F1_metadata.csv', delimiter=',', 
                      encoding='utf-8-sig')

s1f1_md['Image ID'] = s1f1_md['Image ID'].str[:5] + '0' + s1f1_md['Image ID'].str[5:8]

s1f2_md = pd.read_csv('C:/Users/Emily/Documents/S1F2/S1F2_md.csv', delimiter=',', 
                 encoding='utf-8-sig')

all_md = s1f1_md.append(s1f2_md)
all_md.rename(columns={'-': 'Date'}, inplace=True)

In [6]:
all_md.head()

,Date,Recorder,Plate ID,Different strains in each well of a single assay plate? (Y/N),Different compounds in each well of a single assay plate? (Y/N),Compound library ID,Compound Well A,Compound Well B,Compound Well C,Compound Well D,...,Chemotaxis Start (24 hrs format),Chemotaxis End (24 hrs format),Image ID,Scanner Slot:,Scanner Slot,Scanner #,Strain Well A,Strain Well B,Strain Well C,Strain Well D
0,10/25/2021,Ehsan,S1F1_R1_01,NaN,NaN,D1P2,B2,B3,B4,B5,...,NaN,NaN,S1F1_001,1,NaN,3,CX10,CX10,CX10,CX10
1,10/25/2021,Ehsan,S1F1_R1_02,NaN,NaN,D1P2,C2,C3,C4,C5,...,NaN,NaN,S1F1_001,2,NaN,3,CX10,CX10,CX10,CX10
2,10/25/2021,Ehsan,S1F1_R1_03,NaN,NaN,D1P2,D2,D3,D4,D5,...,NaN,NaN,S1F1_001,3,NaN,3,CX10,CX10,CX10,CX10
3,10/25/2021,Ehsan,S1F1_R1_04,NaN,NaN,D1P2,E2,E3,E4,E5,...,NaN,NaN,S1F1_001,4,NaN,3,CX10,CX10,CX10,CX10
4,10/25/2021,Ehsan,S1F1_R1_05,NaN,NaN,D1P2,F2,F3,F4,F5,...,NaN,NaN,S1F1_002,1,NaN,3,CX10,CX10,CX10,CX10


In [7]:
def add_cipd(row):
    if row['Compound library ID'] != 'D5P1':
        pid = row['Compound library ID'][2:4]
    else:
        pid = 'D5P1'
    return pid

all_md['CPID'] = all_md.apply(lambda row: add_cipd(row), axis=1)


In [8]:
all_md['Scanner Slot:'] = all_md['Scanner Slot:'].apply(str)

In [9]:
df_obj = all_md.select_dtypes(['object', 'string'])
all_md = all_md[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
all_md = all_md.fillna('Empty')
all_md.tail()

,Date,Recorder,Plate ID,Compound library ID,Compound Well A,Compound Well B,Compound Well C,Compound Well D,Chemotaxis Start (24 hrs format),Chemotaxis End (24 hrs format),Image ID,Scanner Slot:,Strain Well A,Strain Well B,Strain Well C,Strain Well D,CPID
34,3/17/2022,Ehsan,S1F2_R3_11,D3P1,F2,F3,F4,F5,2:45,3:45,S1F2_009,3,GN960,GN960,GN960,GN960,P1
35,3/17/2022,Ehsan,S1F2_R3_12,D3P1,G2,G3,G4,G5,2:45,3:45,S1F2_009,4,GN960,GN960,GN960,GN960,P1
36,3/21/2022,EMILY,S1F2_R4_1,D4P1,F5,G3,G2,F4,10:55,11:55,S1F2_010,1,GN960,GN960,GN960,GN960,P1
37,3/21/2022,EMILY,S1F2_R4_2,D4P1,E2,C4,E4,B3,10:55,11:55,S1F2_010,2,GN960,GN960,GN960,GN960,P1
38,3/21/2022,EMILY,S1F2_R4_3,D4P2,D4,F2,Empty,Empty,10:55,11:55,S1F2_010,3,GN960,GN960,GN960,GN960,P2


In [10]:
def add_compoundA(row, compound_map):
    if row['Compound Well A'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well A'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"
    
def add_compoundB(row, compound_map ):
    if row['Compound Well B'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well B'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"

def add_compoundC(row, compound_map ):
    if row['Compound Well C'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well C'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"

def add_compoundD(row, compound_map ):
    if row['Compound Well D'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well D'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"


In [11]:
all_md['Compound A'] = all_md.apply(
    lambda row: add_compoundA(row, ckeys), axis=1)

all_md['Compound B'] = all_md.apply(
    lambda row: add_compoundB(row, ckeys), axis=1)
    
all_md['Compound C'] = all_md.apply(
    lambda row: add_compoundC(row, ckeys), axis=1)
    
all_md['Compound D'] = all_md.apply(
    lambda row: add_compoundD(row, ckeys), axis=1)

In [12]:
F1_ia_results = pd.read_csv('C:/Users/Emily/Documents/S1f1/124_ia/S1F1.csv', index_col=0).drop(columns=['Large Object'])
F2_ia_results = pd.read_csv('C:/Users/Emily/Documents/S1F2/S1F2_ia.csv', index_col=0).drop(columns=['Large Object'])
make_ups = pd.read_csv('C:/Users/Emily/Documents/S1F2/S1F2_ia.csv', index_col=0).drop(columns=['Large Object'])
all_ia = F1_ia_results.append(F2_ia_results)


In [13]:
broll = ['S1F1_b010', 'S1F1_b011', 'S1F1_b012']
all_ia= all_ia[~all_ia['File Name'].isin(broll)]
ia_results = all_ia.loc[all_ia['Total Worms'] >= 150]
ia_results.head()

,WellNo,Total Worms,Chemotaxis,Compound,Strain,File Name,Well width,Plate ID
1,1B,199.0,0.233533,NaN,NaN,S1F1_001,3044.0,NaN
2,1C,306.0,0.225352,NaN,NaN,S1F1_001,3044.0,NaN
3,1D,208.0,0.390374,NaN,NaN,S1F1_001,3057.0,NaN
4,2A,282.0,-0.019920,NaN,NaN,S1F1_001,3057.0,NaN
5,2B,239.0,0.171429,NaN,NaN,S1F1_001,3043.0,NaN


In [14]:
def add_PlateID(row, metadata):
    slotID = row['WellNo'][0]
    pid = metadata.loc[
        (metadata['Image ID']==row['File Name']) & 
        (metadata['Scanner Slot:']==slotID)]['Plate ID']

    if len(pid) == 0:
        print(row['File Name'])
        pass
    #print(pid)
    else:
        return pid.values[0]

ia_results['Plate ID'] = ia_results.apply(
    lambda row: add_PlateID(row, all_md), axis=1)

ia_results.head()

C:\ProgramData\Anaconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,WellNo,Total Worms,Chemotaxis,Compound,Strain,File Name,Well width,Plate ID
1,1B,199.0,0.233533,NaN,NaN,S1F1_001,3044.0,S1F1_R1_01
2,1C,306.0,0.225352,NaN,NaN,S1F1_001,3044.0,S1F1_R1_01
3,1D,208.0,0.390374,NaN,NaN,S1F1_001,3057.0,S1F1_R1_01
4,2A,282.0,-0.019920,NaN,NaN,S1F1_001,3057.0,S1F1_R1_02
5,2B,239.0,0.171429,NaN,NaN,S1F1_001,3043.0,S1F1_R1_02


In [15]:
#ia = pd.read_csv('D:/_2021_08_screen/S1fu/ub_md.csv', index_col=0)
#ia.head()

In [16]:
def add_Compound(row, metadata):
    wellID = row['WellNo'][1]

    if wellID == 'A':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound A']
    elif wellID == 'B':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound B']
    elif wellID == 'C':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound C']
    elif wellID == 'D':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound D']
    #print(compound)
    return compound.values[0]

ia_results['Compound'] = ia_results.apply(
    lambda row: add_Compound(row, all_md), axis=1)



C:\ProgramData\Anaconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [17]:
def add_Strain(row, metadata):
    slotID = row['WellNo'][0]
    strain = metadata.loc[
        (metadata['Plate ID']==row['Plate ID']) & 
        (metadata['Scanner Slot:']==slotID)]['Strain Well A ']
    
    if len(strain) == 0:
        return 'Empty'
    else:
        return strain.values[0]

ia_results['Strain'] = ia_results.apply(
    lambda row: add_Strain(row, all_md), axis=1)


C:\ProgramData\Anaconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [18]:
filtered = ia_results.loc[ia_results['Total Worms'] >= 150]
print(len(filtered))

487


In [19]:
filtered = filtered.loc[filtered['Compound'] != 'Empty']

In [20]:
filtered = filtered.sort_values(['Compound','Strain', 'File Name'], ascending=[True, True, True])


In [21]:
S1F1_worm_locs_foldr = plb.Path('C:/Users/Emily/Documents/S1f1/124_ia/')
S1F2_worm_locs_foldr = plb.Path('C:/Users/Emily/Documents/S1F2/')

In [22]:
# 1 inch = 25.4mm
mm = 25.4
# 1200 pixels per 25.4mm
px_mm = 1200/mm

#The following transforms the worm location data so that it is respective to the start zone
# The start zone is at the center of the image; Start Zone = 0mm
#Worms with positive values are closer to the compound, negative values are away from the compound


In [23]:
def get_worm_locs(row, wrms, result_dict, strain): 

    fname = row['File Name']
    wellnum = row['WellNo']

    if row['Strain'] == strain:
        loc_fname =  wrms.joinpath('loc_' + fname + '_' + wellnum + '.csv')

        temp = pd.read_csv(loc_fname)
        compound = row['Compound']
        xs = temp['X']
        #xs = list(temp['centroid-1'])
        if compound in result_dict:
            result_dict[compound] = result_dict[compound].append(xs)
            result_dict[compound].reset_index(inplace=True, drop=True)
            #result_dict[compound] = result_dict[compound]+xs
        else:
            result_dict[compound]=xs

    return result_dict


In [24]:

def all_xs(strn, df, loc):
# Need to create an empty dictionary to hold the values
    results_dict = {}
    compound = ''
    i=0
    
    df2 = df.loc[df['Strain'] == strn]

    for index, row in df2.iterrows():
        if row['Compound'] == compound:
            i += 1
            compound = row['Compound']
            if i < 3:
                pooled = get_worm_locs(row, loc, results_dict, strn)
            else:
                continue
        else:
            i = 0
            compound = row['Compound']
            pooled = get_worm_locs(row, loc, results_dict, strn)

    pooled_df = pd.DataFrame.from_dict(pooled)
    strain_df = pooled_df.apply(lambda x: -(x/px_mm)+32.5)

    strain_df = strain_df.drop(columns = ['Rosmarinic acid', '2-Nonylquinolin-4(1H)-one'])
    strain_df.to_csv('C:/Users/Emily/Documents/S1F2/' + strn + '_xs.csv')
    return strain_df

In [25]:
GNmm_df = all_xs('GN960', filtered, S1F2_worm_locs_foldr)
PRmm_df = all_xs('PR678', filtered, S1F1_worm_locs_foldr)
CXmm_df = all_xs('CX10', filtered, S1F1_worm_locs_foldr)

In [26]:
n2 = pd.read_csv('C:/Users/Emily/Documents/S1/S1_xs3.csv')

In [27]:
cols = list(GNmm_df.columns)
cols.insert(0, cols.pop(cols.index('DMSO')))
print(cols)

['DMSO', '(-)-Huperzine A', '1-octanol', '2,3-Dihydrobenzofuran', '2,5-Dihydroxybenzoic acid', '2-Methyl-1-butanol', '2-nonanone', 'Acetophenone', 'Anisole', 'Camphor', 'Carnosol', 'Cinnamyl Alcohol', 'Coumaran', 'Daucosterol', 'Diacetyl', 'Ellagic acid', 'Ethyl p-methoxycinnamate', 'Ethyl palmitate', 'Furfural', 'Guaiazulene', 'H2O', 'Isoamyl alcohol', 'Isoquinoline', 'L-Mimosine', 'Lapachol', 'Leonurine', 'Limonin', 'Methyl palmitate', 'Oleanolic Acid', 'Paeoniflorin', 'Phenylacetylene', 'Phytol', 'Piperitenone', 'Piperonyl Alcohol', 'Sabinene', 'Salvinorin A Propionate', 'Sinomenine hydrochloride', 'Solasodine', 'Spinosad', 'Thiophene', 'Ursolic acid', 'p-Tolualdehyde', 'α-Phellandrene']


In [29]:
temp = pd.DataFrame()
N2_all = pd.DataFrame()

strains = ['PR678', 'CX10', 'GN960']
refs = ['DMSO', 'H2O']

for s in strains:
    if s == 'CX10':
        xs = CXmm_df
    elif s == 'PR678':
        xs = PRmm_df
    elif s == 'GN960':
        xs = GNmm_df
    for c in cols:
        temp['N2'] = n2[c]
        temp[s] = xs[c]
        db_obj = db.load(temp, idx=(['N2', s]))
        hold = db_obj.mean_diff.results
        hold['Compound'] = c
        N2_all = N2_all.append(hold)
            
N2_all.to_csv('C:/Users/Emily/Documents/S1F2/N2_all.csv')
    

PermissionError: [Errno 13] Permission denied: 'C:/Users/Emily/Documents/S1F2/N2_all.csv'

In [31]:

def mean_diff_calc(df,strain , ref):
    cols = list(df.columns)
    cols.insert(0, cols.pop(cols.index(ref)))
    db_obj = db.load(df, idx=(cols))
    results_df = db_obj.mean_diff.results
    results_df['Strain'] = strain
    #results_df.to_csv('C:/Users/Emily/Documents/S1F2/'+ strain + '_' + ref 
    #                 + '_meandif.csv')
    return results_df

In [32]:
all_strains = pd.DataFrame()

for s in strains:
    for r in refs:
        if s == 'CX10':
            xs = CXmm_df
        elif s == 'PR678':
            xs = PRmm_df
        elif s == 'GN960':
            xs = GNmm_df
        df = mean_diff_calc(xs, s, r)
        all_strains = all_strains.append(df)
#all_strains.to_csv('C:/Users/Emily/Documents/S1F2/all_strains_nullrefs_mdiff.csv')a

In [33]:
all_strains_wide = pd.pivot_table(all_strains, values = ['difference', 'bca_low', 'bca_high', 'pvalue_mann_whitney'], 
                                  index=['test', 'control'], columns = 'Strain').reset_index()

all_strains_wide.columns = [c[1] + c[0] for c in all_strains_wide.columns]



In [34]:
all_strains_wide.head()

,test,control,CX10bca_high,GN960bca_high,PR678bca_high,CX10bca_low,GN960bca_low,PR678bca_low,CX10difference,GN960difference,PR678difference,CX10pvalue_mann_whitney,GN960pvalue_mann_whitney,PR678pvalue_mann_whitney
0,(-)-Huperzine A,DMSO,1.854002,0.675229,3.777346,-1.201574,-1.590813,1.196282,0.282733,-0.434662,2.506552,7.361396e-01,0.732891,3.505104e-04
1,(-)-Huperzine A,H2O,-0.351686,1.815512,0.093510,-3.262607,-0.429063,-2.766574,-1.788521,0.732551,-1.309145,1.634356e-02,0.060494,1.025602e-01
2,1-octanol,DMSO,0.347436,0.852508,-2.707069,-2.655425,-0.936400,-5.293354,-1.173355,-0.040677,-3.953032,2.396278e-01,0.392430,4.766293e-09
3,1-octanol,H2O,-1.803656,2.035567,-6.385948,-4.739255,0.236459,-9.190271,-3.244609,1.126535,-7.768729,1.614153e-05,0.047452,1.973604e-24
4,"2,3-Dihydrobenzofuran",DMSO,11.154475,-0.482385,8.232949,8.303091,-2.399113,5.381876,9.735851,-1.424593,6.791056,2.257249e-33,0.010270,5.056799e-19


In [35]:
n2_dat = pd.read_csv('C:/Users/Emily/Documents/S1/inner3.csv', index_col=0)
n2_dat['Strain'] = 'N2'

n2_dat_wide = pd.pivot_table(n2_dat, values = ['difference', 'bca_low', 'bca_high', 'pvalue_mann_whitney'], 
                                  index=['test', 'control'], columns = 'Strain').reset_index()

n2_dat_wide.columns = [c[1] + c[0] for c in n2_dat_wide.columns]
n2_dat_wide.head()

,test,control,N2bca_high,N2bca_low,N2difference,N2pvalue_mann_whitney
0,(-)-Huperzine A,DMSO,4.772488,1.869578,3.377967,6.207573e-06
1,(-)-Huperzine A,H2O,4.435206,1.392195,2.959551,2.309168e-04
2,1-octanol,DMSO,-5.916856,-8.944625,-7.446258,1.296941e-18
3,1-octanol,H2O,-6.307701,-9.412386,-7.864674,4.284519e-20
4,"2,3-Dihydrobenzofuran",DMSO,10.673937,7.629789,9.194467,1.172357e-28


In [36]:
all_strains_wide = all_strains_wide.merge(n2_dat_wide, on=['test','control'], how='left')
all_strains_wide.to_csv('C:/Users/Emily/Documents/NP_manuscript/Data/all_strains_wide.csv')

In [230]:
GN_sig = N2_all.loc[(N2_all['pvalue_mann_whitney']<.05)&(N2_all['test'] == 'GN960')][['control', 'test', 'Compound', 'difference','pvalue_mann_whitney']].sort_values(by=['difference'])
#GN_sig = GN_sig.rename({'difference':'GN960 Effect Size', 'pvalue_mann_whitney': 'GN960 MWU pvalue'})

PR_sig = N2_all.loc[(N2_all['pvalue_mann_whitney']<.05)&(N2_all['test'] == 'PR678')][['control', 'test', 'Compound','difference' ,'pvalue_mann_whitney']]
CX_sig = N2_all.loc[(N2_all['pvalue_mann_whitney']<.05)&(N2_all['test'] == 'CX10')][['control', 'test', 'Compound', 'difference', 'pvalue_mann_whitney']]

In [231]:
def add_mutant_p(row, d):
    b =  d.loc[d['Compound']==row['Compound']][['difference', 'pvalue_mann_whitney']]
    if len(b) == 0:
        b=['NaN', 'NaN']
        #print(b)
    else:
        return b.values[0].tolist()

GN_sig[['PR678 Effect Size', 'PR678 MWU pvalue']] = GN_sig.apply(
    lambda row: add_mutant_p(row, PR_sig), axis=1, result_type='expand')

GN_sig[['CX10 Effect Size', 'CX10 MWU pvalue']] = GN_sig.apply(
    lambda row: add_mutant_p(row, CX_sig), axis=1, result_type='expand')

GN_sig = GN_sig.rename(columns={'difference':'GN960 Effect Size', 'pvalue_mann_whitney': 'GN960 MWU pvalue'}).drop(columns=['control', 'test'])


In [272]:
comps = all_strains['test'].unique().tolist()


comps.remove('DMSO')
comps.remove('H2O')
comps

['(-)-Huperzine A',
 '1-octanol',
 '2,3-Dihydrobenzofuran',
 '2,5-Dihydroxybenzoic acid',
 '2-Methyl-1-butanol',
 '2-nonanone',
 'Acetophenone',
 'Anisole',
 'Camphor',
 'Carnosol',
 'Cinnamyl Alcohol',
 'Coumaran',
 'Daucosterol',
 'Diacetyl',
 'Ellagic acid',
 'Ethyl p-methoxycinnamate',
 'Ethyl palmitate',
 'Furfural',
 'Guaiazulene',
 'Isoamyl alcohol',
 'Isoquinoline',
 'L-Mimosine',
 'Lapachol',
 'Leonurine',
 'Limonin',
 'Methyl palmitate',
 'Oleanolic Acid',
 'Paeoniflorin',
 'Phenylacetylene',
 'Phytol',
 'Piperitenone',
 'Piperonyl Alcohol',
 'Sabinene',
 'Salvinorin A Propionate',
 'Sinomenine hydrochloride',
 'Solasodine',
 'Spinosad',
 'Thiophene',
 'Ursolic acid',
 'p-Tolualdehyde',
 'α-Phellandrene']

In [276]:
for s in strains:
    for c in comps:
        dmso = all_strains.loc[(all_strains['test'] == c) & (all_strains['control']=='DMSO') & 
                               (all_strains['Strain'] ==  s)][['bca_low', 'bca_high']].values[0].tolist()
        h2o = all_strains.loc[(all_strains['test'] == c) & (all_strains['control']=='H2O') & 
                               (all_strains['Strain'] ==  s)][['bca_low', 'bca_high']].values[0].tolist()

        if (dmso[0] > 0) & (dmso[1]>0)  & (h2o[0] > 0) & (h2o[1] > 0) :
            print('Positive DMSO: ' + c + ' ' + s)
        elif (dmso[0] < 0) & (dmso[1] <0) & (h2o[0] < 0) & (h2o[1] < 0) :
            print('Negative DMSO: ' + c + ' ' + s)

Negative DMSO: 1-octanol PR678
Positive DMSO: 2,3-Dihydrobenzofuran PR678
Negative DMSO: 2-nonanone PR678
Negative DMSO: Acetophenone PR678
Positive DMSO: Anisole PR678
Positive DMSO: Carnosol PR678
Positive DMSO: Diacetyl PR678
Negative DMSO: Furfural PR678
Positive DMSO: Piperitenone PR678
Positive DMSO: 2,3-Dihydrobenzofuran CX10
Positive DMSO: 2-Methyl-1-butanol CX10
Negative DMSO: 2-nonanone CX10
Positive DMSO: Anisole CX10
Positive DMSO: Diacetyl CX10
Negative DMSO: Guaiazulene CX10
Positive DMSO: Isoamyl alcohol CX10
Negative DMSO: Oleanolic Acid CX10
Positive DMSO: Paeoniflorin CX10
Positive DMSO: Phenylacetylene CX10
Positive DMSO: Piperitenone CX10
Negative DMSO: Piperonyl Alcohol CX10
Negative DMSO: Acetophenone GN960
Positive DMSO: Daucosterol GN960
Positive DMSO: Solasodine GN960


array(['(-)-Huperzine A', '1-octanol', '2,3-Dihydrobenzofuran',
       '2,5-Dihydroxybenzoic acid', '2-Methyl-1-butanol', '2-nonanone',
       'Acetophenone', 'Anisole', 'Camphor', 'Carnosol',
       'Cinnamyl Alcohol', 'Coumaran', 'Daucosterol', 'Diacetyl',
       'Ellagic acid', 'Ethyl p-methoxycinnamate', 'Ethyl palmitate',
       'Furfural', 'Guaiazulene', 'H2O', 'Isoamyl alcohol',
       'Isoquinoline', 'L-Mimosine', 'Lapachol', 'Leonurine', 'Limonin',
       'Methyl palmitate', 'Oleanolic Acid', 'Paeoniflorin',
       'Phenylacetylene', 'Phytol', 'Piperitenone', 'Piperonyl Alcohol',
       'Sabinene', 'Salvinorin A Propionate', 'Sinomenine hydrochloride',
       'Solasodine', 'Spinosad', 'Thiophene', 'Ursolic acid',
       'p-Tolualdehyde', 'α-Phellandrene', 'DMSO'], dtype=object)

In [183]:
all_strains_nlref = pd.read_csv('C:/Users/Emily/Documents/S1F2/all_strains_nullrefs_mdiff.csv', 
                                index_col = 0)
all_strains_n2 = pd.read_csv('C:/Users/Emily/Documents/S1F2/N2_all.csv', 
                                index_col = 0)

### Filtering for the same response as DMSO or H2O
<p> The data below represents a loss or reduction in behavioral phenotype for each of the mutant lines</p>

In [11]:


cx10 = all_strains_nlref.loc[(all_strains_nlref['Strain'] == 'CX10') & (all_strains_nlref['bca_low']<0) 
                             & (all_strains_nlref['bca_high']>0)]
print(len(cx10['test'].unique().tolist()))

PR678 = all_strains_nlref.loc[(all_strains_nlref['Strain'] == 'PR678') & (all_strains_nlref['bca_low']<0) 
                             & (all_strains_nlref['bca_high']>0)]
print(len(PR678['test'].unique().tolist()))

GN960 = all_strains_nlref.loc[(all_strains_nlref['Strain'] == 'GN960') & (all_strains_nlref['bca_low']<0) 
                             & (all_strains_nlref['bca_high']>0)]
print(len(GN960['test'].unique().tolist()))

29
23
38


In [184]:
### Generating a dataframe that can made in to a table to report the results above
srtd = pd.read_csv('C:/Users/Emily/Documents/S1F2/sig_mut.csv')
srtd_cmps = srtd['Compound']

In [193]:
strains = ['PR678', 'CX10', 'GN960']


def cat_dat(s, df, df2):
    x = df.loc[(df['Strain'] == s)  & (df['bca_low']<0) 
                             & (df['bca_high']>0)]
    
    c = x['test'].unique().tolist()
    
    y = df2.loc[(df2['test'] == s)  & (df2['bca_low']<0) & (df2['bca_high']>0)]
    
    z= y['Compound'].unique().tolist()

    #print(n2c)
    return c, z


In [194]:
nlref = pd.DataFrame(columns=['Compound', 'GN960', 'PR678', 'CX10'])
nlref['Compound'] = srtd_cmps
nlref

,Compound,GN960,PR678,CX10
0,Isoamyl alcohol,NaN,NaN,NaN
1,2-Methyl-1-butanol,NaN,NaN,NaN
2,Thiophene,NaN,NaN,NaN
3,"2,3-Dihydrobenzofuran",NaN,NaN,NaN
4,Diacetyl,NaN,NaN,NaN
5,Acetophenone,NaN,NaN,NaN
6,Phenylacetylene,NaN,NaN,NaN
7,Paeoniflorin,NaN,NaN,NaN
8,α-Phellandrene,NaN,NaN,NaN
9,Coumaran,NaN,NaN,NaN


In [200]:


for s in strains:
    match_null, match_n2  = cat_dat(s, all_strains_nlref, all_strains_n2)
    for index, row in nlref.iterrows():
        if (row['Compound'] in match_null) & (row['Compound'] not in match_n2):
            row[s] = '*  +'
        elif (row['Compound'] in match_null) & (row['Compound'] in match_n2):
            row[s] = '*'
        elif (row['Compound'] not in match_null) & (row['Compound'] not in match_n2):
            row[s] = '+'
    

In [201]:
nlref.to_csv('C:/Users/Emily/Documents/S1F2/n2andnull.csv')

### Filtering for the same response as N2
<p> The data below indicate that there was not a loss in behavioral phenotype compared to N2 </p>